In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')

In [123]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
data = train
train = train.drop(columns=['PassengerId','Name','Ticket','Cabin'])
# test = test.drop(columns=['PassengerId','Name','Ticket','Cabin'])
train.sample(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
359,1,3,female,NaN,0,0,7.8792,Q
450,0,2,male,36.0,1,2,27.7500,S
130,0,3,male,33.0,0,0,7.8958,C
118,0,1,male,24.0,0,1,247.5208,C
468,0,3,male,NaN,0,0,7.7250,Q


In [62]:
X = train.drop(columns='Survived')
y = train['Survived']
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,S
887,1,female,19.0,0,0,30.0000,S
888,3,female,NaN,1,2,23.4500,S
889,1,male,26.0,0,0,30.0000,C


In [99]:
# Cleaning The Data
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(strategy='mean'),[2]),
    ('impute_embarked',SimpleImputer(strategy='constant',fill_value='ffill'),[6])
],remainder='passthrough')


In [100]:
# One Hot Encoding
from sklearn.preprocessing import OneHotEncoder
trf2 = ColumnTransformer([
    ('ohe_categorical',OneHotEncoder(sparse=False,handle_unknown='ignore',drop='first'),[2,6])
],remainder='passthrough')

In [101]:
# Scaling
from sklearn.preprocessing import MinMaxScaler
trf3 = ColumnTransformer([
    ('scaling',MinMaxScaler(),slice(1,10))
])

In [102]:
# Model selection
from sklearn.linear_model import LogisticRegression
trf4 = LogisticRegression()

In [103]:
from sklearn.pipeline import make_pipeline,Pipeline
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),  
])


In [104]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [105]:
final_data = pipe.fit(X_train,y_train)

In [106]:
y_pred = pipe.predict(X_test)

In [107]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6759776536312849

In [108]:
y_pred

array([0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 0], dtype=int64)

In [120]:
predictions = pipe.predict(test)

In [127]:
final_df = pd.DataFrame(test['PassengerId'])
final_df['Survived'] = predictions
final_df.to_csv("predictions.csv",index=False)